In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [3]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware


class EmailState(AgentState):
  email: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [4]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
  {
    "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
    "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
  },
  config=config
)

In [5]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'no '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know. '
                                                                          'I '
                                                                          'can '
                                                                          'reschedule '
        

In [6]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi John,\n\nno problem—thanks for letting me know. I can reschedule for tomorrow at 10:00 AM or 2:00 PM. Do either of those times work? If not, please suggest a time that suits you and I’ll adjust. Looking forward to our conversation.\n\nBest,\nSeán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi John,\\n\\nno problem—thanks for letting me know. I can reschedule for tomorrow at 10:00 AM or 2:00 PM. Do either of those times work? If not, please suggest a time that suits you and I’ll adjust. Looking forward to our conversation.\\n\\nBest,\\nSeán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='909271fb33cb8c22bb19e4a693e1825f')]


In [7]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

no problem—thanks for letting me know. I can reschedule for tomorrow at 10:00 AM or 2:00 PM. Do either of those times work? If not, please suggest a time that suits you and I’ll adjust. Looking forward to our conversation.

Best,
Seán


# Aprove

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

# Reject

In [9]:
from langgraph.types import Command

response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'no '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know. '
                                                                          'I '
                                                                          'can '
                                                                          'reschedule '
        

In [10]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

no problem—thanks for letting me know. I can reschedule for tomorrow at 10:00 AM or 2:00 PM. Do either of those times work? If not, please suggest a time that suits you and I’ll adjust. Looking forward to our conversation.

Your merciful leader, Seán


# Edit

In [11]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know. '
                                                                          'I '
                                                                          'can '
                                                                          'reschedule '
        